In [234]:
import sys
import time

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
# import seaborn as sns
# sns.set()
from sklearn import tree
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import seaborn as sns
from fuzzytree import FuzzyDecisionTreeClassifier

In [276]:
filenames = {
    "emulab_hdd_merged": "emulab_d460_10Gbps_hdd_merged_V2.csv",
    "emulab_hdd_unmerged": "emulab_d460_10Gbps_hdd_unmerged_V2.csv",

    "utah_ssd_merged": "utah_c6525-25g_25Gbps_ssd_merged_V2.csv",
    "utah_ssd_unmerged": "utah_c6525-25g_25Gbps_ssd_unmerged_V2.csv",

    "wisconsin_ssd_merged": "wisconsin_c220g1-10Gbps_ssd_merged_V2.csv",
    "wisconsin_ssd_unmerged": "wisconsin_c220g1-10Gbps_ssd_unmerged_V2.csv",

    "utah_ssd_merged_1g": "utah_c6525-25g_1Gbps_ssd_merged_V2.csv",
    "utah_ssd_unmerged_1g": "utah_c6525-25g_1Gbps_ssd_unmerged_V2.csv",

    "wisconsin_hdd_ssd_merged": "wisconsin_c220g1-10Gbps_hdd_ssd_merged_V2.csv",
    "wisconsin_hdd-ssd_unmerged": "wisconsin_c220g1-10Gbps_hdd_ssd_unmerged_V2.csv",

    "utah_22_unmerged": "22_utah_c6525-25g_25Gbps_ssd_merged_V2.csv",
    "utah_22_merged": "22_utah_c6525-25g_25Gbps_ssd_unmerged_V2.csv"

}

In [445]:
utah_ssd_merged = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
utah_ssd_merged_1g = pd.read_csv(filenames.get('utah_ssd_merged_1g')).drop(['time_stamp', 'through_put'], axis=1)
utah_22_merged = pd.read_csv(filenames.get('utah_22_merged')).drop(['time_stamp', 'through_put'], axis=1)
emulab_hdd_merged = pd.read_csv(filenames.get('emulab_hdd_merged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_ssd_merged = pd.read_csv(filenames.get('wisconsin_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_ssd_unmerged = pd.read_csv(filenames.get('wisconsin_ssd_unmerged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_hdd_ssd_merged = pd.read_csv(filenames.get('wisconsin_hdd_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
utah_ssd_merged_1g = pd.read_csv(filenames.get('utah_ssd_merged_1g')).drop(['time_stamp', 'through_put'], axis=1)

In [446]:
# features = ['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_active', 'sender_write_bytes', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_req_waittime', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_avg_waittime_md', 'receiver_system_cpu_percent', 'receiver_system_memory_percent', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_avg_rtt_value', 'sender_retrans','sender_segs_in', 'sender_seg_out', 'sender_req_waittime', 'sender_read_bytes', 'sender_write_bytes', 'sender_avg_waittime_md', 'sender_ost_read' , 'sender_ost_write','sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes','receiver_segs_in','receiver_seg_out', 'receiver_req_waittime', 'receiver_read_bytes', 'receiver_write_bytes', 'receiver_ost_read', 'receiver_ost_write', 'receiver_avg_waittime_md', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_req_active', 'sender_write_bytes', 'sender_remote_ost_write_bytes',
#             'receiver_seg_out', 'receiver_segs_in', 'receiver_req_waittime', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write' , 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_seg_out', "sender_retrans", 'sender_avg_waittime_md', 'receiver_segs_in', 'receiver_avg_waittime_md' ,'receiver_write_bytes',]

# features =['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_seg_out', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_vsize', 'sender_rss', 'sender_mem_usage_percentage', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_vsize', 'receiver_rss', 'receiver_mem_usage_percentage', 'receiver_req_active', 'receiver_avg_waittime_md', 'receiver_system_cpu_percent']

# features = ['sender_avg_rtt_value', 'sender_pacing_rate', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_seg_out', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_unacked_value', 'sender_send_buffer_value', 'sender_cpu_usage_percentage', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_waittime', 'sender_req_active', 'sender_read_bytes', 'sender_write_bytes', 'sender_ost_read', 'sender_ost_write', 'sender_pending_read_pages', 'sender_read_RPCs_in_flight', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'sender_system_memory_percent', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_avg_retransmission_timeout_value', 'receiver_seg_out', 'receiver_segs_in', 'receiver_wchar', 'receiver_syscw', 'receiver_read_bytes_io', 'receiver_cpu_usage_percentage', 'receiver_req_waittime', 'receiver_req_active', 'receiver_read_bytes', 'receiver_write_bytes', 'receiver_ost_read', 'receiver_ost_write', 'receiver_pending_read_pages', 'receiver_read_RPCs_in_flight', 'receiver_avg_waittime_md', 'receiver_req_waittime_md', 'receiver_system_cpu_percent', 'receiver_system_memory_percent', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

#19 features
# features = ['sender_seg_out', 'sender_segs_in', 'sender_avg_send_value', 'sender_send_buffer_value', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_active', 'sender_read_bytes', 'sender_ost_read', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

features = ['sender_seg_out', 'sender_avg_send_value', 'sender_tcp_snd_buffer_max', 'sender_read_bytes', 'sender_ost_read', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

In [ ]:
[ 'sender_tcp_snd_buffer_max','receiver_seg_out', 'receiver_read_bytes_io'] # 'sender_req_active', 'sender_segs_in', 'sender_send_buffer_value', 'sender_tcp_snd_buffer_min', 'receiver_req_active'

In [453]:
# 142
emulab_hdd_merged["sender_seg_out"] = emulab_hdd_merged["sender_seg_out"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_seg_out.mean()
emulab_hdd_merged["sender_avg_send_value"] = emulab_hdd_merged["sender_avg_send_value"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_avg_send_value.mean()
emulab_hdd_merged["sender_ost_read"] = emulab_hdd_merged["sender_ost_read"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_ost_read.mean()
emulab_hdd_merged["sender_read_bytes"] = emulab_hdd_merged["sender_read_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_read_bytes.mean()
emulab_hdd_merged["sender_remote_ost_read_bytes"] = emulab_hdd_merged["sender_remote_ost_read_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_remote_ost_read_bytes.mean()
emulab_hdd_merged["sender_remote_ost_write_bytes"] = emulab_hdd_merged["sender_remote_ost_write_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_remote_ost_write_bytes.mean()
emulab_hdd_merged["receiver_segs_in"] = emulab_hdd_merged["receiver_segs_in"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_segs_in.mean()
emulab_hdd_merged["receiver_write_bytes"] = emulab_hdd_merged["receiver_write_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_write_bytes.mean()
emulab_hdd_merged["receiver_remote_ost_read_bytes"] = emulab_hdd_merged["receiver_remote_ost_read_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_remote_ost_read_bytes.mean()
emulab_hdd_merged["receiver_ost_write"] = emulab_hdd_merged["receiver_ost_write"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_ost_write.mean()
emulab_hdd_merged["receiver_remote_ost_write_bytes"] = emulab_hdd_merged["receiver_remote_ost_write_bytes"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_remote_ost_write_bytes.mean()
# emulab_hdd_merged["sender_send_buffer_value"] = emulab_hdd_merged["sender_send_buffer_value"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_send_buffer_value.mean()


In [454]:
# wisconsin_ssd_merged.receiver_write_bytes.mean()/410

wisconsin_ssd_merged["sender_seg_out"] = wisconsin_ssd_merged["sender_seg_out"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_seg_out.mean()
wisconsin_ssd_merged["sender_avg_send_value"] = wisconsin_ssd_merged["sender_avg_send_value"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_avg_send_value.mean()
wisconsin_ssd_merged["sender_ost_read"] = wisconsin_ssd_merged["sender_ost_read"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_ost_read.mean()
wisconsin_ssd_merged["sender_read_bytes"] = wisconsin_ssd_merged["sender_read_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_read_bytes.mean()
wisconsin_ssd_merged["sender_remote_ost_read_bytes"] = wisconsin_ssd_merged["sender_remote_ost_read_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_remote_ost_read_bytes.mean()
wisconsin_ssd_merged["sender_remote_ost_write_bytes"] = wisconsin_ssd_merged["sender_remote_ost_write_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_remote_ost_write_bytes.mean()
wisconsin_ssd_merged["receiver_segs_in"] = wisconsin_ssd_merged["receiver_segs_in"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_segs_in.mean()
wisconsin_ssd_merged["receiver_write_bytes"] = wisconsin_ssd_merged["receiver_write_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_write_bytes.mean()
wisconsin_ssd_merged["receiver_remote_ost_read_bytes"] = wisconsin_ssd_merged["receiver_remote_ost_read_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_remote_ost_read_bytes.mean()
wisconsin_ssd_merged["receiver_ost_write"] = wisconsin_ssd_merged["receiver_ost_write"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_ost_write.mean()
wisconsin_ssd_merged["receiver_remote_ost_write_bytes"] = wisconsin_ssd_merged["receiver_remote_ost_write_bytes"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_remote_ost_write_bytes.mean()
# wisconsin_ssd_merged["sender_send_buffer_value"] = wisconsin_ssd_merged["sender_send_buffer_value"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_send_buffer_value.mean()

In [385]:
print(wisconsin_ssd_merged.receiver_ost_write.mean(), wisconsin_ssd_merged.receiver_ost_write.std())
print(emulab_hdd_merged.receiver_ost_write.mean(), emulab_hdd_merged.receiver_ost_write.std())
print(utah_ssd_merged.receiver_ost_write.mean(), utah_ssd_merged.receiver_ost_write.std())

wisconsin_ssd_merged["receiver_ost_write"] = wisconsin_ssd_merged["receiver_ost_write"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_ost_write.mean()
emulab_hdd_merged["receiver_ost_write"] = emulab_hdd_merged["receiver_ost_write"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_ost_write.mean()
utah_ssd_merged["receiver_ost_write"] = utah_ssd_merged["receiver_ost_write"] / utah_ssd_merged[utah_ssd_merged.label_value == 0].receiver_ost_write.mean()
#
# print(wisconsin_ssd_merged.sender_avg_waittime_md.mean()/410, wisconsin_ssd_merged.sender_avg_waittime_md.std())
# print(emulab_hdd_merged.sender_avg_waittime_md.mean()/142, emulab_hdd_merged.sender_avg_waittime_md.std())
# emulab_hdd_merged[["sender_avg_waittime_md", "label_value"]]
# emulab_hdd_merged.sender_req_waittime_md.mean()
# wisconsin_ssd_merged.groupby('label_value')['sender_seg_out', 'sender_avg_waittime_md', 'receiver_avg_waittime_md'].mean()
# emulab_hdd_merged.groupby('label_value')['sender_avg_waittime_md', 'receiver_avg_waittime_md'].mean()

print(wisconsin_ssd_merged.receiver_ost_write.mean(), wisconsin_ssd_merged.receiver_ost_write.std())
print(emulab_hdd_merged.receiver_ost_write.mean(), emulab_hdd_merged.receiver_ost_write.std())
print(utah_ssd_merged.receiver_ost_write.mean(), utah_ssd_merged.receiver_ost_write.std())
# print(utah_ssd_merged.sender_seg_out.mean(), utah_ssd_merged.sender_seg_out.std())


5676325.3154329145 4616196.176696007
6148683.5600658385 5025424.752579223
5667307.219528374 4701472.103700688
13844.695891299792 11259.015065112213
43300.58845116788 35390.31515900862
0.5890564032170151 0.48866809932028504


In [ ]:
# columns=["label_value", "receiver_avg_waittime_md", "sender_avg_waittime_md", "sender_req_waittime", "sender_system_cpu_percent", "sender_cpu_usage_percentage", "sender_system_memory_percent",
#                            "receiver_cpu_usage_percentage", "receiver_req_waittime", "receiver_system_cpu_percent", "receiver_system_memory_percent",
#                            'sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_retrans', 'sender_ssthresh_value',
#                            'sender_pacing_rate', 'sender_avg_retransmission_timeout_value', 'sender_unacked_value', 'receiver_syscw']

In [456]:
# df = emulab_hdd_merged.copy()
df = wisconsin_ssd_merged.copy()
df = grouped_to_5_cat(df,161)
# df = utah_ssd_merged.copy()
# df = wisconsin_ssd_unmerged.copy()
# print(df.groupby('label_value').size())
# print(df.shape)
# df.loc[df.label_value != 0, "label_value"] = 1
test_df = df.groupby('label_value').sample(40)
# test_df = df.loc[df['label_value']==0].sample(75)
# print(test_df.groupby('label_value').size())
# print(test_df.shape)
# test_df

train_df = df.drop(test_df.index)

# train_df = df
# print(train_df.shape)

y = train_df.label_value
X = train_df[features]#.drop(columns=["label_value", "receiver_avg_waittime_md", "sender_avg_waittime_md", "sender_req_waittime", "sender_system_cpu_percent", "sender_cpu_usage_percentage", "sender_system_memory_percent","receiver_cpu_usage_percentage", "receiver_req_waittime", "receiver_system_cpu_percent", "receiver_system_memory_percent",'sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_retrans', 'sender_ssthresh_value','sender_pacing_rate', 'sender_avg_retransmission_timeout_value', 'sender_unacked_value', 'receiver_syscw', 'receiver_req_waittime_md', 'receiver_avg_retransmission_timeout_value'])#.drop(columns="label_value")[features] #
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(X_train.shape)
# X_train, y_train = RandomUnderSampler(sampling_strategy="all").fit_resample(X_train, y_train)
X_train, y_train = RandomOverSampler(sampling_strategy="all").fit_resample(X_train, y_train)

print(X_train.shape)
# clf = RandomForestClassifier(n_estimators=100)
clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100))
# clf = make_pipeline(StandardScaler(), tree.DecisionTreeClassifier())
# clf = make_pipeline(StandardScaler(), FuzzyDecisionTreeClassifier())
# clf = tree.DecisionTreeClassifier()
# clf = FuzzyDecisionTreeClassifier()
# clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))

# y_pred
y_pred_5cat = change_labels_5(y_pred, 21) #change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21) #change_labels_21(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
# y_pred_5cat[y_pred_5cat[:]==55]
# y_pred_5cat
Counter(y_pred_5cat)

(18006, 14)
(30370, 14)
94.77
94.77


Counter({55: 2065, 1: 1989, 145: 434, 109: 1506, 0: 8})

In [433]:
score = np.round(clf.feature_importances_, 2)

# print(sorted(list(score), reverse=True))
print("Cummulative Importance Score: \n", np.cumsum(sorted(list(score), reverse=True)))
print(Counter(score))
features = []
for i in range(len(score)):
    if score[i] >= 0.02:
        features.append(X.columns[i])
        # print(df.columns[i])
print(features)
print(len(features))


AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

In [457]:
# utah_ssd_merged = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
df_2 = emulab_hdd_merged.copy() # wisconsin_hdd_ssd_merged.copy()#utah_ssd_merged.copy()# wisconsin_ssd_merged.copy() #utah_ssd_merged_1g.copy() #wisconsin_ssd_merged.copy() #
df_2 = grouped_to_5_cat(df_2,161)
# df_2 = wisconsin_ssd_merged.copy() #utah_ssd_merged.copy()#
y_test = df_2.label_value
X_test = df_2[features] #.drop(columns="label_value")##[features] #.drop(columns="label_value")
# X_train, X_test, y_train, y_test = train_test_split(X,y)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))

y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21)#change_labels_21(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))

74.04
74.04


In [6]:
def change_labels_5(y, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # # group read together
    for i in range(1, 55):
        aggregated_labels.update({i: 1})

    # group write together
    for i in range(55, 109):
        aggregated_labels.update({i: 55})
    # group networks together
    for i in range(109, 145):
        aggregated_labels.update({i: 109})
    # group tcp configs together
    for i in range(145, 157):
        aggregated_labels.update({i: 145})
    # group with networks together
    for i in range(157, 161):
        aggregated_labels.update({i: 109})
    y_new = np.array([aggregated_labels[int(i)] for i in y])
    return y_new

def change_labels_21(y, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # group read_congestion_by_sender_sender_ost read levels together
    for i in range(1, 10):
        aggregated_labels.update({i: 1})
    # group read_congestion_by_clients_on_sender_ost read levels together
    for i in range(10, 19):
        aggregated_labels.update({i: 10})
    # group read_congestion_by_receiver_on_receiver_ost read levels together
    for i in range(19, 28):
        aggregated_labels.update({i: 19})
    # group read_congestion_by_clients_on_receiver_ost read levels together
    for i in range(28, 37):
        aggregated_labels.update({i: 28})
    # group read_congestion_by_sender_on_other_ost read levels together
    for i in range(37, 46):
        aggregated_labels.update({i: 37})
    # group read_congestion_by_writer_on_other_ost read levels together
    for i in range(46, 55):
        aggregated_labels.update({i: 46})
    # group write_congestion_by_sender_on_sender_ost read levels together
    for i in range(55, 64):
        aggregated_labels.update({i: 55})
    # group write_congestion_by_clients_on_sender_ost read levels together
    for i in range(64, 73):
        aggregated_labels.update({i: 64})
    # group write_congestion_by_receiver_on_receiver read levels together
    for i in range(73, 82):
        aggregated_labels.update({i: 73})
    # group write_congestion_by_clients_on_receiver read levels together
    for i in range(82, 91):
        aggregated_labels.update({i: 82})
    # group write_congestion_by_sender_on_other_ost read levels together
    for i in range(91, 100):
        aggregated_labels.update({i: 91})
    # group write_congestion_by_receiver_on_other_ost read levels together
    for i in range(100, 109):
        aggregated_labels.update({i: 100})
    # group network_anomaly_network_loss read levels together
    for i in range(109, 113):
        aggregated_labels.update({i: 109})
    # group network_anomaly_network_delay read levels together
    for i in range(113, 117):
        aggregated_labels.update({i: 113})
    # group network_anomaly_network_duplicate read levels together
    for i in range(117, 129):
        aggregated_labels.update({i: 117})
    # group network_anomaly_network_corrupt read levels together
    for i in range(129, 133):
        aggregated_labels.update({i: 129})
    # group network_anomaly_network_reorder read levels together
    for i in range(133, 145):
        aggregated_labels.update({i: 133})
    # group sys_config_tcp_send read levels together
    for i in range(145, 151):
        aggregated_labels.update({i: 145})
    # group sys_config_tcp_receive_buffer read levels together
    for i in range(151, 157):
        aggregated_labels.update({i: 151})
    for i in range(157, 161):
        aggregated_labels.update({i: 157})
    y_new = np.array([aggregated_labels[int(i)] for i in y])
    return y_new

94.25


Counter({55: 2051, 109: 1379, 1: 1915, 145: 418, 0: 79})

In [254]:
score = np.round(clf.feature_importances_, 2)

# print(sorted(list(score), reverse=True))
print("Cummulative Importance Score: \n", np.cumsum(sorted(list(score), reverse=True)))
print(Counter(score))
features = []
for i in range(len(score)):
    if score[i] >= 0.02:
        features.append(X.columns[i])
        # print(df.columns[i])
print(features)
print(len(features))


Cummulative Importance Score: 
 [0.05 0.1  0.14 0.18 0.22 0.26 0.29 0.32 0.35 0.38 0.41 0.44 0.46 0.48
 0.5  0.52 0.54 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7  0.72 0.73 0.74
 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88
 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
 1.   1.  ]
Counter({0.0: 60, 0.01: 28, 0.02: 14, 0.03: 6, 0.04: 4, 0.05: 2})
['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_seg_out', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_waittime', 'sender_req_active', 'sender_ost_read', 'sender_system_cpu_percent', 'sender_remote

In [177]:
y_test = test_df.label_value
X_test = test_df[features] #.drop(columns="label_value")
print(X_test.shape)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
# Counter(y_pred)
y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21)#change_labels_21(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
# y_pred_5cat[y_pred_5cat[:]==55]
# y_pred_5cat
# Counter(y_pred_5cat)

(840, 24)


ValueError: X has 24 features, but StandardScaler is expecting 117 features as input.

In [307]:
new_df = pd.read_csv('./montage_analysis/normal.csv').drop(['time_stamp', 'through_put'], axis=1)
# new_df = pd.read_csv('./montage_analysis/montage_5_instance.csv').drop(['time_stamp', 'through_put'], axis=1)
y_test = new_df.label_value
X_test = new_df[features] #.drop(columns="label_value")#[features] #.drop(columns="label_value")
print(X_test.shape)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
print(Counter(y_pred))
# print(y_test)

y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21)#change_labels(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
print(Counter(y_pred_5cat))
# Counter(y_pred_5cat)
# # new_df.loc[new_df['label_value']==0]
# np.round(X_test.mean(), 2)
# np.round(train_df[train_df.label_value == 0][features].std(), 2) / np.round(X_test.std(), 2)
# train_df.groupby("label_value").receiver_avg_waittime_md.mean()

(75, 20)
0.0
Counter({1: 49, 55: 21, 109: 4, 145: 1})
0.0
Counter({1: 49, 55: 21, 109: 4, 145: 1})


In [225]:
new_df[new_df.receiver_write_bytes <= 310378496.].receiver_write_bytes
# new_df[new_df.sender_cwnd_rate <= 330.].sender_cwnd_rate

53    301989888
54    310378496
Name: receiver_write_bytes, dtype: int64

In [161]:
train_df[train_df.label_value == 0][features]

,sender_avg_rtt_value,sender_cwnd_rate,sender_retrans,sender_ssthresh_value,sender_segs_in,sender_avg_send_value,sender_tcp_snd_buffer_min,sender_tcp_snd_buffer_max,sender_req_active,sender_write_bytes,...,receiver_segs_in,receiver_req_waittime,receiver_req_active,receiver_write_bytes,receiver_ost_write,receiver_avg_waittime_md,receiver_system_cpu_percent,receiver_system_memory_percent,receiver_remote_ost_read_bytes,receiver_remote_ost_write_bytes
9007,0.263,659.0,0.0,548.0,3401.0,29026.1,4096,4194304,1179,0,...,382156.0,10071030,1219,528482304,10071030,508,4.5,1.7,0,528482304
9011,0.312,616.0,0.0,474.0,3075.0,22871.0,4096,4194304,1188,0,...,385019.0,9177842,1186,528482304,9177842,508,4.2,2.0,0,532676608
9012,0.474,639.0,0.0,474.0,3202.0,15616.4,4096,4194304,1188,0,...,386568.0,10169760,1207,528482304,10169760,508,4.5,2.0,0,532676608
9014,0.382,659.0,0.0,474.0,3342.0,19983.9,4096,4194304,1190,0,...,385154.0,9967355,1218,528482304,9967355,508,4.5,2.1,0,528482304
9022,0.283,373.0,0.0,260.0,3830.0,15268.0,4096,4194304,1083,0,...,377159.0,5364040,756,532676608,5364040,508,4.6,1.6,0,545259520
9023,0.585,603.0,0.0,260.0,3544.0,11940.4,4096,4194304,1188,0,...,382615.0,10075130,1222,536870912,10075130,508,4.3,1.7,0,528482304
9027,0.567,557.0,0.0,475.0,2936.0,11379.7,4096,4194304,857,0,...,370055.0,4106501,582,503316480,4106501,508,3.9,1.9,0,532676608
9031,0.526,666.0,0.0,475.0,3213.0,14667.2,4096,4194304,1143,0,...,382616.0,10106007,1213,528482304,10106007,508,4.4,2.1,0,524288000
9034,0.322,681.0,0.0,475.0,3235.0,24499.1,4096,4194304,1179,0,...,383120.0,10022413,1145,499122176,10022413,508,4.4,2.3,0,499122176
9036,0.332,687.0,0.0,475.0,3345.0,23970.5,4096,4194304,1179,0,...,382956.0,10033328,1053,461373440,10033328,508,4.1,2.4,0,457179136


In [106]:
class_names = [str(lbl) for lbl in sorted(Counter(train_df.label_value).keys())]
class_names

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [105]:
# fn=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
# cn=['setosa', 'versicolor', 'virginica']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (120,60), dpi=150)
tree.plot_tree(clf, feature_names = features,
               class_names=class_names,
               filled = True,
               fontsize=8);
fig.savefig('tree_4.pdf')

# tree.plot_tree(clf, feature_names=features, fontsize=12)

Error in callback <function flush_figures at 0x7fc6c56d9e50> (for post_execute):


KeyboardInterrupt: 

In [59]:
# utah_ssd_merged.loc[utah_ssd_merged['label_value']==0]

,sender_avg_rtt_value,sender_pacing_rate,sender_cwnd_rate,sender_avg_retransmission_timeout_value,sender_byte_ack,sender_seg_out,sender_retrans,sender_ssthresh_value,sender_segs_in,sender_avg_send_value,...,receiver_setattr_md,receiver_read_page_md,receiver_unlink_md,receiver_avg_dsack_dups_value,receiver_avg_reord_seen,receiver_system_cpu_percent,receiver_system_memory_percent,receiver_remote_ost_read_bytes,receiver_remote_ost_write_bytes,label_value
11117,0.303,22064.8,627.0,201.0,520.100807,377341.0,0.0,140.0,1888.0,23970.9,...,0,0,0,0.0,0.0,2.5,2.6,0,499122176,0
11118,0.427,22064.8,816.0,201.0,547.453789,396506.0,0.0,140.0,1469.0,22137.1,...,0,0,0,0.0,0.0,2.6,2.6,0,499122176,0
11119,0.356,22064.8,912.0,201.0,544.445404,394149.0,0.0,140.0,1404.0,29675.9,...,0,0,0,0.0,0.0,2.7,2.7,0,494927872,0
11120,0.285,22064.8,992.0,201.0,544.851418,395750.0,0.0,140.0,1481.0,40320.4,...,0,0,0,0.0,0.0,2.6,2.7,0,486539264,0
11121,0.400,22064.8,1044.0,201.0,547.148582,395435.0,0.0,140.0,1499.0,30234.2,...,0,0,0,0.0,0.0,2.6,2.8,0,482344960,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11293,0.394,594.7,755.0,201.0,545.555634,395799.0,221.0,738.0,1291.0,22197.8,...,0,0,0,0.0,0.0,2.7,3.6,0,511705088,0
11294,0.286,594.7,820.0,201.0,550.444366,398585.0,0.0,738.0,1565.0,33212.9,...,0,0,0,0.0,0.0,2.6,3.6,0,507510784,0
11295,0.339,594.7,875.0,201.0,544.000000,394159.0,0.0,738.0,1469.0,29899.7,...,0,0,0,0.0,0.0,2.6,3.7,0,503316480,0
11296,0.325,594.7,946.0,201.0,544.000000,394227.0,0.0,738.0,1274.0,33718.4,...,0,0,0,0.0,0.0,2.6,3.8,0,478150656,0


In [243]:
def grouped_to_5_cat(df, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # # group read together
    for i in range(1, 55):
        aggregated_labels.update({i: 1})

    # group write together
    for i in range(55, 109):
        aggregated_labels.update({i: 55})
    # group networks together
    for i in range(109, 145):
        aggregated_labels.update({i: 109})
    # group tcp configs together
    for i in range(145, 157):
        aggregated_labels.update({i: 145})
    # group with networks together
    for i in range(157, 161):
        aggregated_labels.update({i: 109})
    y = [aggregated_labels[int(i)] for i in df[df.columns[len(df.columns) - 1]].values]
    df["label_value"] = y
    return df

In [19]:
df = pd.read_csv(filenames.get('wisconsin_ssd_unmerged')).drop(['time_stamp', 'through_put'], axis=1)
# df = grouped_to_5_cat(df,161)
df.shape
Counter(df.label_value)

Counter({98: 148,
         38: 151,
         31: 137,
         59: 159,
         27: 138,
         141: 163,
         63: 158,
         80: 158,
         123: 139,
         136: 154,
         147: 142,
         13: 154,
         120: 145,
         125: 146,
         24: 139,
         26: 143,
         124: 143,
         9: 163,
         57: 158,
         15: 158,
         47: 143,
         4: 169,
         29: 123,
         114: 155,
         1: 168,
         22: 137,
         73: 142,
         7: 159,
         131: 160,
         122: 139,
         54: 153,
         36: 136,
         83: 137,
         49: 143,
         132: 153,
         121: 146,
         157: 161,
         126: 145,
         39: 152,
         127: 142,
         105: 151,
         128: 146,
         56: 154,
         37: 150,
         154: 157,
         70: 152,
         11: 161,
         52: 155,
         60: 157,
         133: 147,
         103: 138,
         66: 160,
         42: 154,
         35: 139,
         116

In [316]:
t = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
t[t.label_value == 0].receiver_avg_waittime_md

11117    149
11118    149
11119    149
11120    149
11121    149
        ... 
11293    158
11294    158
11295    158
11296    158
11297    158
Name: receiver_avg_waittime_md, Length: 181, dtype: int64